In [ ]:
DATASET_ROOT_RID = 'YOUR_DIR_HERE'

In [ ]:
import h5py
from IPython.display import Image, display
import imageio
import numpy as np  
import matplotlib.pyplot as plt  
import yaml


def display_gif_from_images(images, fps=10, filename='temp.gif'):  
    imageio.mimsave(filename, images, format='GIF', fps=fps)  
    display(Image(filename=filename))


def print_group_structure(group, indent=''):  
    for key in group.keys():  
        item = group[key]
        if isinstance(item, h5py.Dataset):  
            if key == 'shape':
                print(f"{indent}- {key}: {item[()]}")  
            else:
                print(f"{indent}- {key}: {item.shape}")  

        elif isinstance(item, h5py.Group):
            print(f"{indent}- {key}: Group")
            print_group_structure(item, indent + '  ')
def accumulate_delta_actions(actions):
    for i in range(1, len(actions)):
        actions[i] += actions[i-1]

  
def plot_3d_points_with_gradient_colors(points):  
    x = points[:, 0]  
    y = points[:, 1]  
    z = points[:, 2]  

    num_points = len(points)
    colors = plt.cm.viridis(np.arange(num_points) / (num_points - 1))

    fig = plt.figure()  
    ax = fig.add_subplot(111, projection='3d')  
    ax.scatter(x, y, z, c=colors)  

    ax.set_xlabel('X-axis')  
    ax.set_ylabel('Y-axis')  
    ax.set_zlabel('Z-axis')  
    plt.show()  

def get_statistics(all_actions: np.array):
    length = all_actions.shape[-1]
    def format_statistic(data):
        s = '['
        for i in range(length):
            s += str(data[i])
            if i != length-1:
                s += ','
        s += ']'
        return s
    print(f'min: {format_statistic(all_actions.min(axis=0))}')
    print(f'max: {format_statistic(all_actions.max(axis=0))}')
    print(f'mean: {format_statistic(all_actions.mean(axis=0))}')
    print(f'std: {format_statistic(all_actions.std(axis=0))}')

In [ ]:
ds_cfgs = yaml.safe_load(open('./data_preprocessing/data_cfg.yaml', 'r'))

def ds_cfg_generator():
    for k,v in ds_cfgs.items():
        if k[0] != '_':
            yield (k, v)

ds_cfg_iter = ds_cfg_generator()

In [ ]:
dataset_name, cfg = next(ds_cfg_iter)

In [ ]:

f_path = f'/{DATASET_ROOT_RID}/{dataset_name}.hdf5'
outer_key = cfg['action']['outer_key']
inner_keys = cfg['action']['inner_keys']
f = h5py.File(f_path, 'r')
image_key = list(cfg['image'].keys())[0]

episode = f['episodes/episode_1/']
actions = []
for inner_key in inner_keys:
    v = episode[outer_key][inner_key][()]
    if v.dtype == bool or len(v.shape) == 1:
        v = np.array(v, dtype=np.float32).reshape(-1, 1)
    actions.append(v)
actions = np.concatenate(actions, axis=1)

In [ ]:
images = episode['observation'][image_key][()]
display_gif_from_images(images)

In [ ]:
f.close()

In [ ]:
dataset_name = 'fractal20220817_data'
f_path = f'/{DATASET_ROOT_RID}/{dataset_name}.hdf5'

f = h5py.File(f_path, 'r')

info = f['shape_type_info']
print_group_structure(info)
episodes = f['episodes']
print(f'episode count: {episodes["length"][()]}')

In [ ]:
def get_episode_action(one_episode):
    actions = []
    for inner_key in inner_keys:
        v = one_episode[outer_key][inner_key][()]
        if v.dtype == bool:
            v = np.array(v, dtype=np.float32).reshape(-1, 1)
        actions.append(v)
    actions = np.concatenate(actions, axis=1)
    return actions

In [ ]:
# action:
outer_key = 'action'
inner_keys = ['world_vector', 'rotation_delta', 'gripper_closedness_action']
one_episode = episodes['episode_1']

actions = get_episode_action(one_episode)

images = one_episode['observation']['image'][()]

In [ ]:
# this is desired to be applied on actions as if action is delta, actions should be accumulated to represent a whole trajectory (to be displayed)
accumulate_delta_actions(actions[:, :3])

In [ ]:
plot_3d_points_with_gradient_colors(actions)

In [ ]:
display_gif_from_images(images)

In [ ]:
import tqdm
num_episodes = episodes['length'][()]
episode_lengthes = [0] * num_episodes
for i in tqdm.trange(num_episodes):
    episode_lengthes[i] = int(episodes[f'episode_{i}']['length'][()])

In [ ]:

all_actions = np.zeros(shape=(sum(episode_lengthes), actions.shape[-1], ), dtype=np.float32)

begin_idx = 0
for i in tqdm.trange(num_episodes):
    all_actions[begin_idx: begin_idx+episode_lengthes[i]] = get_episode_action(episodes[f'episode_{i}'])
    begin_idx += episode_lengthes[i]

get_statistics(all_actions)